In [ ]:
import os
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

class_names = ['0', '1', '2', '3', 
               '4', '5', '6', '7', 
               '8', '9', 'A', 'B', 
               'C', 'D', 'E', 'F', 
               'G', 'H', 'K', 'L', 
               'M', 'N', 'O', 'P', 
               'R', 'S', 'T', 'U', 
               'V', 'X', 'Y', 'Z']

datagen = ImageDataGenerator(
    rescale=1.0/255, 
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='training',
    class_mode='sparse'
)

validation_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='validation',
    class_mode='sparse'
)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (5,5), activation='elu', 
                           padding='same',
                           input_shape=(28, 12, 3)),
    tf.keras.layers.Conv2D(128, (5,5), activation='elu',
                           padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3,3), activation='elu', 
                           padding='same'),
    tf.keras.layers.Conv2D(256, (3,3), activation='elu',
                           padding='same'),
    tf.keras.layers.Conv2D(128, (3,3), activation='elu',
                           padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='elu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
class stopCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}) :
        if (logs.get('accuracy') > 0.98) :
            print('\nReached 98% accuracy so stopping training')
            self.model.stop_training = True

In [6]:
callback = stopCallback()
model.fit(train_generator, epochs=1000, callbacks=[callback])

Train for 87 steps
Epoch 1/1000
87/87 [==============================] - 166s 2s/step - loss: 3.2076 - accuracy: 0.1243
Epoch 2/1000
87/87 [==============================] - 125s 1s/step - loss: 2.0797 - accuracy: 0.4303
Epoch 3/1000
87/87 [==============================] - 159s 2s/step - loss: 1.4339 - accuracy: 0.6061
Epoch 4/1000
87/87 [==============================] - 150s 2s/step - loss: 1.1798 - accuracy: 0.6744
Epoch 5/1000
87/87 [==============================] - 133s 2s/step - loss: 0.9994 - accuracy: 0.7178
Epoch 6/1000
87/87 [==============================] - 128s 1s/step - loss: 0.9097 - accuracy: 0.7403
Epoch 7/1000
87/87 [==============================] - 133s 2s/step - loss: 0.7029 - accuracy: 0.7964
Epoch 8/1000
87/87 [==============================] - 140s 2s/step - loss: 0.6142 - accuracy: 0.8219
Epoch 9/1000
87/87 [==============================] - 172s 2s/step - loss: 0.6022 - accuracy: 0.8265
Epoch 10/1000
30/87 [=========>....................] - ETA: 1:56 - loss:

KeyboardInterrupt: 

In [ ]:
model.evaluate(validation_generator, verbose=1)

In [ ]:
path1 = "../Testing_set/greenparkingchar"
path2 = "../Testing_set/greenparking2char"

all_path = [path1, path2]

final = "../charLabel/"

i = 1

for path in all_path:
    for file in os.listdir(path) :
        image_mp= mpimg.imread(path + '/' + file) / 255
        image_mp = np.expand_dims(image_mp, axis=0)
        shutil.move(path + '/' + file, final + class_names[np.argmax(model.predict(image_mp))] + '/' + file)
        i += 1
        if i == 100:
            break
    if i == 100:
        break